In [33]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from pydantic import BaseModel, Field, model_validator

# deployment_name = "gpt-4o-mini"
# embedding_model = "gpt-4o-mini"
# openai_api_version="2024-07-18"

system_prompt = ("""
Você é um assistente de IA que extrai dados de documentos.
Você deve extrair informações específicas e retorná-las em um formato estruturado.

INSTRUÇÕES CRÍTICAS:
- EVITE DUPLICADOS: Nunca inclua itens duplicados em qualquer lista.
- SEJA CONCISO: Mantenha cada item breve e direto ao ponto.
- VALIDE: Cada informação deve estar explicitamente declarada no texto; não faça suposições.

Por favor, extraia:
- Liste as informações do Documento

Texto do documento:
{context}

"""
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

# https://cog-openai-stage.openai.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview
llm = AzureChatOpenAI(azure_deployment="gpt-4o-mini", api_version="2024-08-01-preview", temperature=0.0)

In [146]:
from enum import Enum
from pydantic import BaseModel, Field, ValidationError, field_validator

class GarantiaEnum(str, Enum):
    """Enumeração para tipos de garantia"""
    AF_IMOVEL = "Alienação Fiduciária Imóvel"
    AF_MAQ_EQ = "Alienação Fiduciária Maq&Eq"
    PENHOR = "Penhor"
    AF_QUOTAS = "Alienação Fiduciária quotas"
    AVAL_PJ = "Aval (PJ)"
    AVAL_PF = "Aval (PF)"
    FIANCA_BANCARIA = "Fiança Bancária"
    CF_RECEBIVEIS = "CF Recebiveis"
    CF_CARTAO = "CF Cartão"
    CF_DIVIDENDOS = "CF Dividendos"
    CF = "Cessão Fiduciária"
    SERIE_MEZ = "Série Mez"
    SERIE_SUB = "Série Sub"
    COOBRIGACAO = "Coobrigação"
    FUNDO_RESERVA = "Fundo de Reserva"
    FUNDO_DESPESAS = "Fundo de Despesas"
    FUNDO_JUROS = "Fundo de Juros"
    FUNDO_OBRAS = "Fundo de Obras"
    OUTROS_FUNDOS = "Outros Fundos"
    OUTRAS = "Outras"

class GarantiaModel(BaseModel):
    garantia: GarantiaEnum = Field(default=GarantiaEnum.OUTRAS)

    @field_validator("garantia", mode="before")
    @classmethod
    def validate_garantia(cls, value):
        # If the value is not valid, return the default value
        if value not in GarantiaEnum.__members__.values():
            return GarantiaEnum.OUTRAS
        return value

# Test cases
try:
    print(GarantiaModel(garantia="Penhor"))  # Output: garantia='Penhor'
except ValidationError as e:
    print(e)

try:
    print(GarantiaModel(garantia="Invalid Value"))  # Output: garantia='Outras' (default value)
except ValidationError as e:
    print(e)

print(GarantiaModel())  # Output: garantia='Outras' (default value)


garantia=<GarantiaEnum.PENHOR: 'Penhor'>
garantia=<GarantiaEnum.OUTRAS: 'Outras'>
garantia=<GarantiaEnum.OUTRAS: 'Outras'>


In [153]:
from typing import List, Optional
from datetime import datetime, date
from enum import Enum
from pydantic_core import core_schema

class TipoEnum(str, Enum):
    """CERTIFICADOS DE RECEBIVEIS IMOBILIARIOS"""
    CRI = 'CRI'
    """CERTIFICADOS DE RECEBIVEIS DO AGRONEGOCIO"""
    CRA = 'CRA'
    """DEBENTURES"""
    DEB = 'DEB'
    """CERTIFICADOS DE RECEBIVEIS"""
    CR = 'CR'

class RegiaoEnum(str, Enum):
    """Região Sul"""
    S = 'S'
    """Região Sudeste"""
    SE = 'SE'
    """Região Centro Oeste"""
    CO = 'CO'
    """Região Nordeste"""
    NE = 'NE'
    """Região Norte"""
    N = 'N'

class LocalidadeModel(BaseModel):
    """Localização da Parte"""
    cidade: str = Field(description="Cidade da Parte")
    estado: str = Field(description="Estado da Parte")
    regiao: RegiaoEnum = Field(description="Região do estado da Parte")

class SecuritizadoraEmissoraModel(BaseModel):
    """Parte nomeadas e qualificadas como Securitizadora ou Emissora"""
    cnpj: str = Field(description="Número do CNPJ da Securitizadora ou Emissora. Formatos permitidos: ##.###.###/####-##")
    nome: str = Field(description="Nome da empresa ou Razão Social da Securitizadora ou Emissora")

class AgenteFiduciarioModel(BaseModel):
    """Parte nomeadas e qualificadas como Agente Fiduciário"""
    cnpj: str = Field(description="Número do CNPJ do Agente Fiduciário. Formatos permitidos: ##.###.###/####-##")
    nome: str = Field(description="Nome da empresa ou Razão Social do Agente Fiduciário")

class DevedoraModel(BaseModel):
    """Parte nomeadas e qualificadas como Devedores ou Devedoras"""
    cpf_cnpj: str = Field(description="Número do CPF ou CNPJ do Devedor. Formatos permitidos: ##.###.###/####-## ou ###.###.###-##")
    nome: str = Field(description="Nome do Devedor")
    localidade: LocalidadeModel = Field(description="Localidade do Devedor")

class CedenteModel(BaseModel):
    """Parte nomeadas e qualificadas como Cedentes"""
    cpf_cnpj: str = Field(description="Número do CPF ou CNPJ do Cedente. Formatos permitidos: ##.###.###/####-## ou ###.###.###-##")
    nome: str = Field(description="Nome do Cedente")
    localidade: LocalidadeModel = Field(description="Localidade do Devedor")

class AvalistaFiadorModel(BaseModel):
    """Parte nomeadas e qualificadas como Avalista ou Fiador"""
    cpf_cnpj: str = Field(description="Número do CPF ou CNPJ do Avalista ou Fiador. Formatos permitidos: ##.###.###/####-## ou ###.###.###-##")
    nome: str = Field(description="Nome do Avalista ou Fiador")
    localidade: LocalidadeModel = Field(description="Localidade do Devedor")

class IndexadorEnum(str, Enum):
    IPCA = 'IPCA'
    DI = 'DI'
    PRE = 'Pré'
    INCC = 'INCC'
    PTAX = 'PTAX'
    IGPM = 'IGPM'
    OUTROS = 'Outros'

class PeriodoTaxaEnum(str, Enum):
    """Ao Ano"""
    AA = 'aa'
    """Ao Mês"""
    AM = 'am'

class RemuneracaoModel(BaseModel):
    """Informações sobre a remuneração do Ativo"""
    indexador: Optional[IndexadorEnum] = Field(description="Indexador da Remuneração do CRI ou CRA")

    variacao: float = Field(description="Variação percentual acumulada do indexador da Remuneração do CRI ou CRA em %", examples=[100], default=100.0)

    taxa_adicional_spread: Optional[float] = Field(description="Taxa Adicional pré fixada ou Spread acrescida na Remuneração do CRI ou CRA em %", examples=[1.0, 5.5, 10.0])

    periodo_taxa: Optional[PeriodoTaxaEnum] = Field(description="Periodo da Taxa Adicional da Remuneração do CRI ou CRA")

    @field_validator("indexador", mode="before")
    @classmethod
    def validate_indexador(cls, value):
        # If the value is not valid, return the default value
        if value not in IndexadorEnum.__members__.values():
            return IndexadorEnum.OUTROS
        return value

class LastroEnum(str, Enum):
    """Cédula de Crédito Bancário"""
    CCB = "CCB"
    """Certificado de Direitos Creditórios do Agronegócio"""
    CDCA = "CDCA"
    """Compra e Venda"""
    COMPRA_VENDA = "COMPRA_VENDA"
    """Cédula do Produto Rural"""
    CPR_F = "CPR-F"
    """Debênture"""
    DEBENTURE = "Debênture"
    """Duplicata"""
    DUPLICATA = "Duplicata"
    """Financiamento Imobiliário"""
    FINANCIAMENTO_IMOBILIARIO = "Financiamento Imobiliário"
    """Letra Financeira"""
    LETRA_FINANCEIRA = "Letra Financeira"
    """Nota Comercial"""
    NOTA_COMERCIAL = "Nota Comercial"
    """Nota Promissória"""
    NOTA_PROMISSORIA = "Nota Promissória"
    """Outros Recebíveis"""
    OUTROS_RECEBIVEIS = "Outros Recebíveis"
    """Pro Soluto"""
    PRO_SOLUTO = "Pro Soluto"
    """Outros Lastros"""
    OUTROS = "Outros"

class ClasseSerieEnum(str, Enum):
    """Senior"""
    SENIOR = "Senior"
    """Subordinada"""
    SUBORDINADA = "Subordinada"

class SerieModel(BaseModel):
    """Informações das Series"""
    numero_serie: str = Field(description="Número da série em sua forma apenas numérica. Caso seja uma série única, retorne UNI.",
                              examples=['1', '2', '3', 'UNI'])

    classe_serie: Optional[ClasseSerieEnum] = Field(description="Classe da série.")

    lastro: Optional[LastroEnum] = Field(description="Lastro da série")

    desc_lastro: Optional[str] = Field(description="Descrição do Lastro da série")

    vlr_unitario_serie: Optional[float] = Field(description="Valor Unitário da Série")

    qt_serie: Optional[int] = Field(description="Quantidade da Série")

    vlr_serie: Optional[float] = Field(description="Valor da Série")

    remuneracao: Optional[RemuneracaoModel] = Field(description="Remuneração da Série (CRI ou CRA)")

    @field_validator("lastro", mode="before")
    @classmethod
    def validate_lastro(cls, value):
        # If the value is not valid, return the default value
        if value not in LastroEnum.__members__.values():
            return LastroEnum.OUTROS
        return value

class CoordenadorLiderModel(BaseModel):
    """Parte nomeadas e qualificadas como Coordenador Lider"""
    cnpj: str = Field(description="Número do CNPJ do Coordenador Líder. Formatos permitidos: ##.###.###/####-##")
    nome: str = Field(description="Nome da empresa ou Razão Social do Coordenador Líder")

class EstruturadorModel(BaseModel):
    """Parte nomeadas e qualificadas como Estruturador"""
    cnpj: str = Field(description="Número do CNPJ do Estruturador. Formatos permitidos: ##.###.###/####-##")
    nome: str = Field(description="Nome da empresa ou Razão Social do Estruturador")

class ResponsavelGarantiaModel(BaseModel):
    """Parte nomeadas e qualificadas como Responsável"""
    cpf_cnpj: str = Field(description="Número do CPF ou CNPJ do Responsável da Garantia. Formatos permitidos: ##.###.###/####-## ou ###.###.###-##")
    nome: str = Field(description="Nome do Responsável da Garantia")

class GarantiaEnum(str, Enum):
    """Alienação Fiduciária Imóvel"""
    AF_IMOVEL = "Alienação Fiduciária Imóvel"
    """Alienação Fiduciária Maq&Eq"""
    AF_MAQ_EQ = "Alienação Fiduciária Maq&Eq"
    """Penhor"""
    PENHOR = "Penhor"
    """Alienação Fiduciária quotas"""
    AF_QUOTAS = "Alienação Fiduciária quotas"
    "Aval (PJ)"
    AVAL_PJ = "Aval (PJ)"
    """Aval (PF)"""
    AVAL_PF = "Aval (PF)"
    """Fiança Bancária"""
    FIANCA_BANCARIA = "Fiança Bancária"
    """Cessão Fiduciária Recebiveis"""
    CF_RECEBIVEIS = "CF Recebiveis"
    """Cessão Fiduciária Cartão"""
    CF_CARTAO = "CF Cartão"
    """Cessão Fiduciária Dividendos"""
    CF_DIVIDENDOS = "CF Dividendos"
    """Cessão Fiduciária"""
    CF = "Cessão Fiduciária"
    """Série Mez"""
    SERIE_MEZ = "Série Mez"
    """Série Sub"""
    SERIE_SUB = "Série Sub"
    """Coobrigação"""
    COOBRIGACAO = "Coobrigação"
    """Fundo de Reserva"""
    FUNDO_RESERVA = "Fundo de Reserva"
    """Fundo de Despesas"""
    FUNDO_DESPESAS = "Fundo de Despesas"
    """Fundo de Juros"""
    FUNDO_JUROS = "Fundo de Juros"
    """Fundo de Obras"""
    FUNDO_OBRAS = "Fundo de Obras"
    """Outros Fundos"""
    OUTROS_FUNDOS = "Outros Fundos"
    """Outras"""
    OUTRAS = "Outras"

class GarantiaModel(BaseModel):
    """Informações das Garantias"""
    garantia: Optional[GarantiaEnum] = Field(description="Nome da Garantia")

    desc_garantia: Optional[str] = Field(description="Descrição da Garantia")

    responsavel: Optional[ResponsavelGarantiaModel] = Field(description="Responsavel da Garantia")

    @field_validator("garantia", mode="before")
    @classmethod
    def validate_garantia(cls, value):
        # If the value is not valid, return the default value
        if value not in GarantiaEnum.__members__.values():
            return GarantiaEnum.OUTRAS
        return value

In [154]:
class AssembleiaModel(BaseModel):
    """Informações importantes da Assembleia"""
    emissora: SecuritizadoraEmissoraModel = Field(description="Securitizadora ou Emissora")

    agente_fiduciario: Optional[AgenteFiduciarioModel] = Field(description="Agente Fiduciário")

    tipo: Optional[TipoEnum] = Field(description="Tipo da Emissão")

    numero_emissao: str = Field(description="Número da emissão em sua forma apenas numérica.")

    numero_serie: List[str] = Field(description="Lista Única de Séries da Emissão em sua forma apenas numérica. Caso seja uma série única, retorne UNI")

    dt_hr_assembleia: Optional[datetime] = Field(description="data e hora da assembleia")

    numero_convocacao: int = Field(description="número de convocação da assembleia nesta data", default=1)

    fl_assembleia_deliberada: Optional[bool] = Field(description="A assembleia teve a Ordem do Dia deliberada?")

    fl_assembleia_nao_instalada: Optional[bool] = Field(description="A assembleia não foi instalada?")

In [155]:
class TermoSecModel(BaseModel):
    """Informações importantes do Termo de Securitização"""
    emissora: SecuritizadoraEmissoraModel = Field(description="Securitizadora ou Emissora")

    agente_fiduciario: Optional[AgenteFiduciarioModel] = Field(description="Agente Fiduciário")

    devedores: List[DevedoraModel] = Field(description="Lista Única de Devedores ou Devedoras")

    cedentes: List[CedenteModel] = Field(description="Lista Única de Cedentes")

    avalistas_fiadores: List[AvalistaFiadorModel] = Field(description="Lista Única de Avalistas ou Fiadores")

    coordenador_lider: Optional[CoordenadorLiderModel] = Field(description="Coordenador Lider")

    estruturador: Optional[EstruturadorModel] = Field(description="Estruturador")

    numero_aditamento: int = Field(description="Número do aditamento. Caso não seja um aditamento retorne 0", default=0)

    tipo: Optional[TipoEnum] = Field(description="Tipo da Emissão")

    numero_emissao: str = Field(description="Número da emissão em sua forma apenas numérica.")

    numero_serie: List[str] = Field(description="Lista Única de Séries da Emissão em sua forma apenas numérica. Caso seja uma série única, retorne UNI")

    series: List[SerieModel] = Field(description="Lista Única de Séries da Emissão. Garanta que todas as séries sejam declaradas.")

    garantias: List[GarantiaModel] = Field(description="Garantias da Emissão")

    dt_emissao: Optional[date] = Field(description="data da emissão")

    dt_vencimento: Optional[date] = Field(description="data do vencimento")

    numero_quorum_minimo_primeira_convocacao: Optional[str] = Field(description="Número quórum mínimo de instalação de assembleia em primeira convocação com a presença de Titulares ou Investidores em Circulação. Retorne o mais simples possível.")

    numero_quorum_minimo_segunda_convocacao: Optional[str] = Field(description="Número quórum mínimo de instalação de assembleia em segunda convocação com a presença de Titulares ou Investidores em Circulação. Retorne o mais simples possível.")

    vlr_unitario_emissao: Optional[float] = Field(description="Valor Unitário da Emissão ou da Oferta")

    qt_total_emissao: Optional[int] = Field(description="Quantidade Total da Emissão ou da Oferta")

    vlr_total_emissao: Optional[float] = Field(description="Valor Total da Emissão ou da Oferta")

In [156]:
class EscrituraModel(BaseModel):
    """Informações importantes da Escritura"""
    emissora: SecuritizadoraEmissoraModel = Field(description="Securitizadora ou Emissora")

    agente_fiduciario: Optional[AgenteFiduciarioModel] = Field(description="Agente Fiduciário")

    avalistas_fiadores: List[AvalistaFiadorModel] = Field(description="Lista Única de Avalistas ou Fiadores")

    coordenador_lider: Optional[CoordenadorLiderModel] = Field(description="Coordenador Lider")

    estruturador: Optional[EstruturadorModel] = Field(description="Estruturador")

    numero_aditamento: int = Field(description="Número do aditamento. Caso não seja um aditamento retorne 0", default=0)

    tipo: Optional[TipoEnum] = Field(description="Tipo da Emissão")

    numero_emissao: str = Field(description="Número da emissão em sua forma apenas numérica.")

    numero_serie: List[str] = Field(description="Lista Única de Séries da Emissão em sua forma apenas numérica. Caso seja uma série única, retorne UNI")

    series: List[SerieModel] = Field(description="Lista Única de Séries da Emissão")

    garantias: List[GarantiaModel] = Field(description="Garantias da Emissão")

    dt_emissao: Optional[date] = Field(description="data da emissão")

    dt_vencimento: Optional[date] = Field(description="data do vencimento")

    numero_quorum_minimo_primeira_convocacao: Optional[str] = Field(description="Número quórum mínimo de instalação de assembleia em primeira convocação com a presença de Titulares ou Investidores em Circulação. Retorne o mais simples possível.")

    numero_quorum_minimo_segunda_convocacao: Optional[str] = Field(description="Número quórum mínimo de instalação de assembleia em segunda convocação com a presença de Titulares ou Investidores em Circulação. Retorne o mais simples possível.")

    vlr_unitario_emissao: Optional[float] = Field(description="Valor Unitário da Emissão ou da Oferta")

    qt_total_emissao: Optional[int] = Field(description="Quantidade Total da Emissão ou da Oferta")

    vlr_total_emissao: Optional[float] = Field(description="Valor Total da Emissão ou da Oferta")

In [157]:
def format_docs(file_path):
    f = open(file_path,"r")
    return f.read()

In [158]:
def invoke(model, file_path):
    llm_with_schema = llm.bind_tools([model])
    # llm_with_schema = llm.with_structured_output(model)

    chain = prompt | llm_with_schema

    result_llm = chain.invoke({"context": format_docs(file_path), "input": "Extraia as informações"})
    pydantic_object = model.model_validate(result_llm.tool_calls[0]["args"])
    return pydantic_object

In [151]:
# FILE_PATH = '../files/BRPVSCCRI3Y2-ASS31012025V01-000829323.txt'
FILE_PATH = '../files/BRASTECRI0C3-ASS04022025V01-000830908.txt'
# FILE_PATH = '../files/23A1772203-ASS04022025V01-000831063.txt'
result = invoke(AssembleiaModel, FILE_PATH)
result.model_dump_json()

'{"emissora":{"cnpj":"10.608.405/0001-60","nome":"Bari Securitizadora S.A."},"agente_fiduciario":{"cnpj":"22.610.500/001-88","nome":"Vórtx Distribuidora de Títulos e Valores Mobiliários LTDA."},"tipo":"CRI","numero_emissao":"1","numero_serie":["70"],"dt_hr_assembleia":"2025-02-04T14:00:00","numero_convocacao":1,"fl_assembleia_deliberada":false,"fl_assembleia_nao_instalada":true}'

## Teste Documentos de Termo de Securitização

In [161]:
# FILE_PATH = '../files/831141.txt'
# FILE_PATH = '../files/24F1234491-TDS07062024V01-000675608.txt'
# FILE_PATH = '../files/797444.txt'
FILE_PATH = '../files/018759000101010.txt'
# FILE_PATH = '../files/819915.txt'
# FILE_PATH = '../files/CRA02400DL4-TDS17122024V01-000803501.txt'

result = invoke(TermoSecModel, FILE_PATH)
result.model_dump_json()

'{"emissora":{"cnpj":"03.767.538/0001-14","nome":"BRAZILIAN SECURITIES COMPANHIA DE SECURITIZAÇÃO"},"agente_fiduciario":{"cnpj":"36.113.876/0001-91","nome":"OLIVEIRA TRUST DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS S.A."},"devedores":[{"cpf_cnpj":"12.098.466/0001-50","nome":"JAY PARTICIPAÇÕES IMOBILIÁRIAS LTDA.","localidade":{"cidade":"Florianópolis","estado":"SC","regiao":"S"}}],"cedentes":[{"cpf_cnpj":"08.117.803/0001-32","nome":"CFL - PARTICIPAÇÕES E INCORPORAÇÕES LTDA.","localidade":{"cidade":"Porto Alegre","estado":"RS","regiao":"S"}}],"avalistas_fiadores":[],"coordenador_lider":null,"estruturador":null,"numero_aditamento":0,"tipo":"CRI","numero_emissao":"1","numero_serie":["333","334"],"series":[{"numero_serie":"333","classe_serie":"Senior","lastro":"Outros","desc_lastro":"Cédulas de Crédito Imobiliário","vlr_unitario_serie":10376011.44,"qt_serie":1,"vlr_serie":10376011.44,"remuneracao":{"indexador":"IGPM","variacao":100.0,"taxa_adicional_spread":12.6825,"periodo_taxa":"aa"}}

## Teste Documentos de Escritura de Debentures

In [53]:
FILE_PATH = "../files/676706.txt"
result = invoke(EscrituraModel, FILE_PATH)
result.model_dump_json()

'{"emissora":{"cnpj":"00.864.214/0001-06","nome":"ENERGISA S.A."},"agente_fiduciario":{"cnpj":"17.343.682/0003-08","nome":"PENTÁGONO S.A. DISTRIBUIDORA DE TÍTULOS E VALORES MOBILIÁRIOS"},"avalistas_fiadores":[],"coordenador_lider":null,"estruturador":null,"numero_aditamento":0,"tipo":"DEB","numero_emissao":"11","numero_serie":["UNI"],"series":[{"numero_serie":"UNI","classe_serie":"Senior","lastro":"Debênture","vlr_unitario_serie":1000.0,"qt_serie":500000,"vlr_serie":500000000.0,"remuneracao":{"indexador":"IPCA","variacao":100.0,"taxa_adicional_spread":0.5,"periodo_taxa":"aa"}}],"garantias":[],"dt_emissao":"2019-04-15","dt_vencimento":"2026-04-15","numero_quorum_minimo_primeira_convocacao":"50% + 1","numero_quorum_minimo_segunda_convocacao":"1/3","vlr_unitario_emissao":1000.0,"qt_total_emissao":500000,"vlr_total_emissao":500000000.0}'